# Import Packages and Files

In [8]:
import torch.nn as nn
from functionalities import dataloader as dl
from functionalities import evaluater as ev
from functionalities import filemanager as fm
from functionalities import trainer as tr
from architecture import RotNet as RN

# Load Dataset

In [9]:
trainset, testset, classes = dl.load_cifar("./datasets")

Files already downloaded and verified
Files already downloaded and verified


In [10]:
trainloader, validloader, testloader = dl.make_dataloaders(trainset, testset, 0, 128)

# Initialize Loss Criterion

In [11]:
criterion = nn.CrossEntropyLoss()

# Train RotNet for Rotation Task and Classifiers on Feature Maps 

In [12]:
# set rot classes
rot_classes = ['original', '90 rotation', '180 rotation', '270 rotation']

## 3 Block RotNet

In [13]:
# initialize network
net_block3 = RN.RotNet(num_classes=4, num_conv_block=3, add_avg_pool=False)

In [ ]:
# train network
rot_block3_loss_log, rot_block3_valid_accuracy_log, rot_block3_test_accuracy_log, rot_block3_max_accuracy, \
rot_block3_best_epoch = tr.adaptive_learning([0.1, 0.02, 0.004, 0.0008], [60, 120, 160, 200], 0.9, 5e-4, net_block3, 
                                             criterion, trainloader, None, testloader, rot=['90', '180', '270'])

In [ ]:
# train NonLinearClassifiers on feature map of net_3block
block3_loss_log, block3_valid_accuracy_log, block3_test_accuracy_log, block3_max_accuracy, block3_best_epoch = \
tr.train_all_blocks(3, 10, [0.1, 0.02, 0.004, 0.0008], [20, 40, 45, 100], 0.9, 5e-4, net_block3, criterion, trainloader,
                    None, testloader) 

In [ ]:
# train ConvClassifiers on feature map of net_3block
conv_block3_loss_log, conv_block3_valid_accuracy_log, conv_block3_test_accuracy_log, conv_block3_max_accuracy, \
conv_block3_best_epoch = tr.train_all_blocks(3, 10, [0.1, 0.02, 0.004, 0.0008], [35, 70, 85, 100], 0.9, 5e-4, net_block3, 
                                            criterion, trainloader, None, testloader, use_ConvClassifier=True) 

In [ ]:
# rename files
fm.add_block_to_name(3, [100, 200], list(rot_block3_best_epoch) + block3_best_epoch + conv_block3_best_epoch)

## 4 Block RotNet

In [ ]:
# initialize network
net_block4 = RN.RotNet(num_classes=4, num_conv_block=4, add_avg_pool=False)

In [ ]:
# train network
rot_block4_loss_log, rot_block4_valid_accuracy_log, rot_block4_test_accuracy_log, rot_block4_max_accuracy, \
rot_block4_best_epoch = tr.adaptive_learning([0.1, 0.02, 0.004, 0.0008], [60, 120, 160, 200], 0.9, 5e-4, net_block4, 
                                             criterion, trainloader, None, testloader, rot=['90', '180', '270'])

In [ ]:
# train NonLinearClassifiers on feature map of net_3block
block4_loss_log, block4_valid_accuracy_log, block4_test_accuracy_log, block4_max_accuracy, block4_best_epoch = \
tr.train_all_blocks(4, 10, [0.1, 0.02, 0.004, 0.0008], [20, 40, 45, 100], 0.9, 5e-4, net_block4, criterion, trainloader,
                    None, testloader) 

In [ ]:
# train ConvClassifiers on feature map of net_3block
conv_block4_loss_log, conv_block4_valid_accuracy_log, conv_block4_test_accuracy_log, conv_block4_max_accuracy, \
conv_block4_best_epoch = tr.train_all_blocks(4, 10, [0.1, 0.02, 0.004, 0.0008], [35, 70, 85, 100], 0.9, 5e-4, net_block4, 
                                            criterion, trainloader, None, testloader, use_ConvClassifier=True) 

In [ ]:
# rename files
fm.add_block_to_name(4, [100, 200], list(rot_block4_best_epoch) + block4_best_epoch + conv_block4_best_epoch)

## 5 Block RotNet

In [ ]:
# initialize network
net_block5 = RN.RotNet(num_classes=4, num_conv_block=5, add_avg_pool=False)

In [ ]:
# train network
rot_block5_loss_log, rot_block5_valid_accuracy_log, rot_block5_test_accuracy_log, rot_block5_max_accuracy, \
rot_block5_best_epoch = tr.adaptive_learning([0.1, 0.02, 0.004, 0.0008], [60, 120, 160, 200], 0.9, 5e-4, net_block5, 
                                             criterion, trainloader, None, testloader, rot=['90', '180', '270'])

In [ ]:
# train NonLinearClassifiers on feature map of net_3block
block5_loss_log, block5_valid_accuracy_log, block5_test_accuracy_log, block5_max_accuracy, block5_best_epoch = \
tr.train_all_blocks(5, 10, [0.1, 0.02, 0.004, 0.0008], [20, 40, 45, 100], 0.9, 5e-4, net_block5, criterion, trainloader,
                    None, testloader) 

In [ ]:
# train ConvClassifiers on feature map of net_3block
conv_block5_loss_log, conv_block5_valid_accuracy_log, conv_block5_test_accuracy_log, conv_block5_max_accuracy, \
conv_block5_best_epoch = tr.train_all_blocks(5, 10, [0.1, 0.02, 0.004, 0.0008], [35, 70, 85, 100], 0.9, 5e-4, net_block5, 
                                            criterion, trainloader, None, testloader, use_ConvClassifier=True) 

In [ ]:
# rename files
fm.add_block_to_name(5, [100, 200], list(rot_block5_best_epoch) + block5_best_epoch + conv_block5_best_epoch)

## 5 Block RotNet with Average Pooling after ConvBlock 3

In [ ]:
# initialize network
net_block5_avg = RN.RotNet(num_classes=4, num_conv_block=5, add_avg_pool=True)

In [ ]:
# train network
rot_block5_avg_loss_log, rot_block5_avg_valid_accuracy_log, rot_block5_avg_test_accuracy_log, \
rot_block5_avg_max_accuracy, rot_block5_avg_best_epoch = tr.adaptive_learning([0.1, 0.02, 0.004, 0.0008], 
    [60, 120, 160, 200], 0.9, 5e-4, net_block5_avg, criterion, trainloader, None, testloader, rot=['90', '180', '270'])

In [ ]:
# train NonLinearClassifiers on feature map of net_3block
block5_avg_loss_log, block5_avg_valid_accuracy_log, block5_avg_test_accuracy_log, block5_avg_max_accuracy, \
block5_best_epoch = tr.train_all_blocks(5, 10, [0.1, 0.02, 0.004, 0.0008], [20, 40, 45, 100], 0.9, 5e-4, net_block5_avg, 
                                        criterion, trainloader, None, testloader) 

In [ ]:
# train ConvClassifiers on feature map of net_3block
conv_block5_avg_loss_log, conv_block5_avg_valid_accuracy_log, conv_block5_avg_test_accuracy_log, \
conv_block5_avg_max_accuracy, conv_block5_avg_best_epoch = tr.train_all_blocks(5, 10, [0.1, 0.02, 0.004, 0.0008], \
    [35, 70, 85, 100], 0.9, 5e-4, net_block5_avg, criterion, trainloader, None, testloader, use_ConvClassifier=True) 

In [ ]:
# rename files
fm.add_block_to_name(5, [100, 200], list(rot_block5_avg_best_epoch) + block5_avg_best_epoch + conv_block5_avg_best_epoch)

# Supervised NIN

Note: In the code of the paper a 3 convolutional block RotNet was used for the classification task.

In [ ]:
# initialize networks
net_class = RN.RotNet(num_classes=10, num_conv_block=3, add_avg_pool=False)

In [ ]:
# train 3 block RotNet on classification task
class_NIN_loss_log, class_NIN_valid_accuracy_log, class_NIN_test_accuracy_log, class_NIN_max_accuracy, \
class_NIN_best_epoch = tr.adaptive_learning([0.1, 0.02, 0.004, 0.0008], [60, 120, 160, 200], 0.9, 5e-4, net_class, 
                                            criterion, trainloader, None, testloader)

# Test Accuracy